In [1]:
!pip install datasets
!pip install trl
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import wandb

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
)
from datasets import Dataset
import os, torch, json, wandb, subprocess
from sklearn.model_selection import train_test_split
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
)

import torch.nn as nn
from trl import SFTTrainer

import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import BitsAndBytesConfig

from tqdm.notebook import tqdm
import random

import re
import logging
import torch
from tqdm.notebook import tqdm
from transformers import StoppingCriteria, StoppingCriteriaList, TextStreamer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Inference

In [4]:



finetuned_model = "/content/drive/MyDrive/dacon/ko-gemma-2-9b-it_AUG:1x_EPOCH:3/checkpoint-3000"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4비트 양자화 적용
    bnb_4bit_compute_dtype=torch.float16,  # 연산을 float16에서 수행
    bnb_4bit_use_double_quant=True,  # 추가적인 양자화 적용
)

model = AutoModelForCausalLM.from_pretrained(
        finetuned_model,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        attn_implementation='eager',
        quantization_config=quantization_config
)

model.config.use_cache = True

tokenizer = AutoTokenizer.from_pretrained(finetuned_model,
                                          trust_remote_code = True
                                          )

test = pd.read_csv('/content/drive/MyDrive/dacon/open/test.csv', encoding = 'utf-8-sig')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/852 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00010.safetensors:   0%|          | 0.00/1.92G [00:00<?, ?B/s]

model-00002-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00008-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00009-of-00010.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [5]:

import pandas as pd

test_result = []
# 로그 설정
logging.basicConfig(filename='run.log', level=logging.INFO, format='%(asctime)s - %(message)s')


# 전체 테스트 데이터 추론 수행
for idx, test_i in enumerate(tqdm(test["input"], desc="추론 진행 중")):

    # 모델 입력 메시지 생성
    messages = [
        {
            "role": "user",
            "content": "다음은 읽기 어렵게 변형된 한글 문장입니다. 모든 문장의 길이, 띄어쓰기, 특수문자는 그대로이며, 한글 글자만 바뀌었습니다. 원래 문장으로 복원해 주세요.\n\n{}".format(test_i)
        }
    ]

    # 모델 입력 프롬프트 생성
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # 모델 생성 수행
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(model.device)

    # 모델 생성 수행
    outputs = model.generate(
        input_ids,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95,
        max_new_tokens=1024,
    )

    # 결과 디코딩
    decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # "model" 이후의 텍스트만 추출
    match = re.search(r"model\s*(.*)", decoded_output, re.DOTALL)
    if match:
        output_text = match.group(1).strip()
    else:
        output_text = decoded_output.strip()

    # "## 원래 문장: " 제거 (필요한 경우)
    output_text = output_text.replace("원래 문장으로 복원해 주세요. ", "").strip()

    # 결과 저장
    test_result.append(output_text)

    # 로그 기록: 추론된 결과를 run.log에 저장
    log_message = f"샘플 {idx+1}\n난독화 문장: {test_i}\n복호화 문장: {output_text}"
    logging.info(log_message)

추론 진행 중:   0%|          | 0/1689 [00:00<?, ?it/s]

The 'batch_size' attribute of HybridCache is deprecated and will be removed in v4.49. Use the more precisely named 'self.max_batch_size' attribute instead.


KeyboardInterrupt: 

In [ ]:
submission = pd.read_csv('./data/sample_submission.csv', encoding = 'utf-8-sig')
submission['output'] = test_result
submission.to_csv('./submission/ko-gemma-2-9b-it_AUG-1x_STEP-3000.csv', index = False, encoding = 'utf-8-sig')
